# Градиентные бустинги и сравнение популярных реализаций

Подготовил: Сабалевский Сергей

План:

1. Вспомним теорию бустингов на примере Adaboost
2. Перейдем рассмотрению градиентного бустинга, какие улучшения бывают
3. Сравним различия реализаций градиентного бустинга. 
4. Сравним их результаты на примере одного датасета:
 - LightGBM
 - CatBoost
 - XGBoost

In [34]:
from sklearn import datasets as skdata # генерация примеров
from sklearn import tree as sktree     # реализация алгоритмов с деревьями решений
from sklearn import base as skbase     # базовые классы с реализацией API
from sklearn import utils as skutils   # полезные вспомогательные функции для модели
from sklearn import metrics as skmetrics

from pprint import pprint

from IPython.display import Image, display_svg, SVG
from dtreeviz.trees import dtreeviz

import scipy.optimize as sciopt
import numpy as np
import pandas as pd

import plotly.graph_objects as go      # визуализация
import plotly.subplots as subp         # для подграфиков

## Бустинги

Это процедура последовательного построения композиции алгоритмов, когда каждый следующий алгоритм стремится компенсировать недостатки композиции всех предыдущих. Если коротко, то это жадный алгоритм построения композиции алгоритмов машинного обучения.

Математическая запись:\
$a(x) = sign(\sum_{t=1}^T \alpha_t b_t(x))$ - заданный алгоритм\
$Q_T(a,X) = \sum_{i=1}^n [y_i\sum_{t=i}^T \alpha_t b_t(x_i)<0]$ - функционал ошибки

для упрощения задачи минимизации функционала можно применить две эвристики:
-  При добавлении в композицию слагаемого $\alpha_tb_t(x)$ оптимизируется только базовый алгоритм $b_t$ и коэффициент при нём $α_t$, а все предыдущие слагаемые $α_1b_1(x), . . . , α_{t−1}b_{t−1}(x)$ полагаются фиксированными. 
- Пороговая функция потерь $Q_T$ апроксимируется сверху непрерывно дифференцируемой. \
$Q_T(a,X) = \sum_{i=1}^n exp(-y_i\sum_{t=i}^{T-1} \alpha_t b_t(x_i)) * e^{-y_i\alpha_Tb_T(x_i)}$

Далее рассмотри конкретный алгоритм __Ada boost classifier__  на примере бинарной классификации:

Достоинства алгоритма:
- Хорошая обобщающая способность. В реальных задачах (не всегда, но часто)
удаётся строить композиции, превосходящие по качеству базовые алгоритмы.
Обобщающая способность может улучшаться (в некоторых задачах) по мере
увеличения числа базовых алгоритмов.
- Простота реализации
- Накладные расходы бустинга невелики. Время построения композиции практически полностью определяется временем обучения базовых алгоритмов.
- Возможность идентифицировать выбросы. 


Недостатки:
- AdaBoost склонен к переобучению при наличии значительного уровня шума
в данных. Экспоненциальная функция потерь слишком сильно увеличивает веса «наиболее трудных» объектов, на которых ошибаются многие базовые алгоритмы.
- AdaBoost требует достаточно длинных обучающих выборок.
- Жадная стратегия последовательного добавления приводит к построению
неоптимального набора базовых алгоритмов.
- Бустинг может приводить к построению громоздких композиций, состоящих
из сотен алгоритмов. Такие композиции исключают возможность содержательной интерпретации, требуют больших объёмов памяти для хранения базовых
алгоритмов и существенных затрат времени на вычисление классификаций.

Как можно улушить алгоритм?
- фильтрация выбросов
- валидационная выборка
- при накоплении ошибки возвращаться назад и переобучать базовые алгоритмы в композции
- после обучения композиции можно провести оптимизацию весов голосования с помощью стандартных методов построения разделяющей поверхности.

# Градиентные бустинги

Первой реализацией бустинга, которая имела большой успех в применении, был Адаптивный бустинг, или сокращенно AdaBoost.

Потом пришло время обобщений.

Статистическая структура представляет собой задачу численной оптимизации, цель которой состоит в том, чтобы минимизировать потери модели путем добавления слабых учащихся с использованием процедуры градиентного спуска.

Обобщение позволило использовать произвольные дифференцируемые функции потерь, расширив метод за пределы задач бинарной классификации для поддержки регрессии, многоклассовой классификации и многого другого.


### Как работает градиентный бустинг

Градиентный бустинг включает в себя три элемента:

1. Функция потерь, подлежащая оптимизации.
2. Слабый ученик, чтобы делать прогнозы.
3. Аддитивная модель для добавления слабых учащихся, чтобы минимизировать функцию потерь.

#### 1. Loss Function

Используемая функция потерь зависит от типа решаемой проблемы.

Он должен быть дифференцируемым, но поддерживаются многие стандартные функции потерь, и вы можете определить свои собственные.

Например, регрессия может использовать квадратическую ошибку, а классификация может использовать логарифмические потери.

Преимущество фреймворка градиентного бустинга заключается в том, что для каждой функции потерь, которую может потребоваться использовать, не нужно выводить новый алгоритм бустинга, вместо этого это достаточно общая структура, в которой можно использовать любую дифференцируемую функцию потерь.

#### 2. Weak Learner

Деревья решений используются в качестве слабого ученика в градиентном бустинге.

В частности, используются деревья регрессии, которые выводят реальные значения для разбиений и выходные данные которых могут быть сложены вместе, позволяя добавлять выходные данные последующих моделей и “корректировать” остатки в прогнозах.

Деревья строятся жадным образом, выбирая лучшие точки разделения на основе показателей чистоты, таких как Джини, или для минимизации потерь.

Первоначально, например, в случае AdaBoost, использовались очень короткие деревья решений, которые имели только одно разделение, называемое пнем решения. Большие деревья можно использовать, как правило, с уровнями от 4 до 8.

Обычно слабые учащиеся ограничиваются определенными способами, такими как максимальное количество слоев, узлов, расщеплений или конечных узлов.

Это делается для того, чтобы учащиеся оставались слабыми, но все еще могли быть построены жадным образом.

#### 3. Additive Model

Деревья добавляются по одному, и существующие деревья в модели не изменяются.

Процедура градиентного спуска используется для минимизации потерь при добавлении деревьев.

Традиционно градиентный спуск используется для минимизации набора параметров, таких как коэффициенты в уравнении регрессии или веса в нейронной сети. После вычисления ошибки или потери веса обновляются, чтобы минимизировать эту ошибку.

Вместо параметров у нас есть слабые подмодели учащихся или, более конкретно, деревья решений. После вычисления потерь, чтобы выполнить процедуру градиентного спуска, мы должны добавить в модель дерево, которое уменьшает потери (т.е. Следовать градиенту). Мы делаем это, параметризуя дерево, затем изменяем параметры дерева и движемся в правильном направлении (уменьшая остаточные потери.

Обычно этот подход называется функциональным градиентным спуском или градиентным спуском с функциями.

Выходные данные для нового дерева затем добавляются к выходным данным существующей последовательности деревьев в попытке исправить или улучшить конечный результат модели.

Фиксированное количество деревьев добавляется или обучение прекращается, как только потери достигают приемлемого уровня или больше не улучшаются во внешнем наборе данных проверки.

### Улучшения в базовом градиентном бустинге


Градиентный бустинг - это жадный алгоритм, который может быстро подогнать обучающий набор данных.

Он может извлечь выгоду из методов регуляризации, которые штрафуют различные части алгоритма и в целом повышают производительность алгоритма за счет уменьшения переобучения.

В этом разделе мы рассмотрим 4 усовершенствования базового градиентного бустинга:

- Tree Constraints
- Shrinkage (Weighted Updates, аналогия learning rate)
- Random sampling (Stochastic Gradient Boosting)
- Penalized Learning (L1, L2 regularization)

#### Tree Constraints

Хорошая общая эвристика заключается в том, что чем более ограничено создание дерева, тем больше деревьев вам понадобится в модели, и наоборот, чем меньше отдельных деревьев, тем меньше деревьев потребуется.

Некоторые ограничения на деревья:

- **Количество деревьев**, как правило, добавление большего количества деревьев в модель может быть очень медленным для перенастройки. Совет состоит в том, чтобы продолжать добавлять деревья до тех пор, пока не будет наблюдаться никакого дальнейшего улучшения.
- **Глубина дерева**, более глубокие деревья являются более сложными деревьями, и более короткие деревья предпочтительнее. Как правило, лучшие результаты наблюдаются с 4-8 уровнями.
- **Количество узлов или количество листьев**, например глубина, это может ограничить размер дерева, но не ограничивается симметричной структурой, если используются другие ограничения.
- **Количество наблюдений за разбиение** накладывает минимальное ограничение на объем обучающих данных в обучающем узле, прежде чем можно будет рассмотреть разбиение

### Перейдем к примерам

<img src='https://miro.medium.com/max/3076/1*i0CA9ho0WArOj-0UdpuKGQ.png' alignment='center'/>

### Структурные различия в LightGBM, XGBoost и Catboost

LightGBM использует новую технику односторонней выборки на основе градиента (Gradient-based One-Side Sampling GOSS) для фильтрации экземпляров данных для нахождения значения разделения, в то время как XGBoost использует предварительно отсортированный алгоритм и алгоритм на основе гистограммы для вычисления наилучшего разделения.

#### pre-sorting splitting

- Для каждого узла проходимся по всем признакам
- Для каждого признака отсортирем экземпляры по значению признака
- Используйте линейное сканирование, чтобы решить, как лучше разделить информацию по этому признаку.
- Возьмем лучшее решение для разделения по всем функциям

Таким образом, XGBoost не использует никаких методов взвешенной выборки, что делает его процесс разделения медленнее по сравнению с GOSS и MVS.

#### GOSS

Основное допущение, принятое здесь, состоит в том, что выборки с обучающими экземплярами с небольшими градиентами имеют меньшую ошибку обучения и уже хорошо обучены.

Чтобы сохранить одинаковое распределение данных, при вычислении прироста информации GOSS вводит постоянный множитель для экземпляров данных с небольшими градиентами. Таким образом, GOSS обеспечивает хороший баланс между сокращением числа экземпляров данных и сохранением точности для изученных деревьев решений.

#### Catboost и его MVS

Catboost предлагает новый метод, называемый выборкой с минимальной дисперсией (MVS), который представляет собой взвешенную версию стохастического градиентного бустинга. В этом методе взвешенная выборка происходит на уровне дерева, а не на уровне разделения. Наблюдения для каждого дерева бустинга отбираются таким образом, чтобы максимизировать точность разделения оценок.

#### Catboost и небрежные (oblivious) деревья решений

Чтобы вырастить сбалансированное дерево. Одни и те же функции используются для создания левых и правых разделений (split) на каждом уровне дерева.

По сравнению с классическими деревьями, небрежные деревья более эффективны при реализации на процессоре и просты в обучении.

### Сходство в гиперпараметрах

<img src='https://miro.medium.com/max/3400/1*A0b_ahXOrrijazzJengwYw.png' width=1000 height=700 alignment='center'/>

### Реализация методов на датасете

Воспользуемся набором данных Kaggle о задержках рейсов за 2015 год, поскольку он имеет как категориальные, 
так и числовые характеристики. Имея примерно 5 миллионов строк, этот набор данных будет полезен для 
оценки производительности с точки зрения скорости и точности настроенных моделей для каждого типа повышения. 
Будем использовать 10% подмножества этих данных ~ 500 тыс. строк.

Ниже приведены признаки, используемые для моделирования:
    
- MONTH, DAY, DAY_OF_WEEK: data type int
- AIRLINE and FLIGHT_NUMBER: data type int
- ORIGIN_AIRPORT and DESTINATION_AIRPORT: data type string
- DEPARTURE_TIME: data type float
- ARRIVAL_DELAY: this will be the target and is transformed into boolean variable indicating delay of more than 10 minutes
- DISTANCE and AIR_TIME: data type float

In [40]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn import metrics
import lightgbm as lgb
import catboost as cb

In [41]:
dataset = pd.read_csv("../data/flights.csv")
dataset.info()

C:\Anaconda3\envs\classification2021\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning:

Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [42]:
data = dataset.sample(frac=0.1, random_state=10)

data = data[["MONTH", "DAY", "DAY_OF_WEEK", "AIRLINE", "FLIGHT_NUMBER", "DESTINATION_AIRPORT",
             "ORIGIN_AIRPORT", "AIR_TIME", "DEPARTURE_TIME", "DISTANCE", "ARRIVAL_DELAY"]]
data.dropna(inplace=True)

data["ARRIVAL_DELAY"] = (data["ARRIVAL_DELAY"] > 10)*1

cols = ["AIRLINE", "FLIGHT_NUMBER", "DESTINATION_AIRPORT", "ORIGIN_AIRPORT"]
for item in cols:
    data[item] = data[item].astype("category").cat.codes + 1

train, test, y_train, y_test = train_test_split(data.drop(["ARRIVAL_DELAY"], axis=1), data["ARRIVAL_DELAY"],
                                                random_state=10, test_size=0.25)

In [43]:
len(train), len(test)

(428504, 142835)

In [44]:
def auc_from_pred(m, train, test):
    return (
        metrics.roc_auc_score(y_train, m.predict(train)),
        metrics.roc_auc_score(y_test, m.predict(test)),
    )

In [45]:
def auc_from_proba(m, train, test):
    return (
        metrics.roc_auc_score(y_train, m.predict_proba(train)[:, 1]),
        metrics.roc_auc_score(y_test, m.predict_proba(test)[:, 1]),
    )

### XGBoost

In [46]:
model = xgb.XGBClassifier(
    max_depth=50,
    min_child_weight=1,
    n_estimators=200,
    n_jobs=-1,
    verbose=1,
    learning_rate=0.16,
)
model.fit(train, y_train)

C:\Anaconda3\envs\classification2021\lib\site-packages\xgboost\sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



[16:35:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:35:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.16, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbose=1,
              verbosity=None)

In [47]:
auc_xgb = auc_from_proba(model, train, test)
print(np.round(auc_xgb, 3))

[1.    0.789]


#### Light GBM

In [48]:
d_train = lgb.Dataset(train, label=y_train, free_raw_data=False)
params = {"max_depth": 50, "learning_rate": 0.1, "num_leaves": 900, "n_estimators": 300}

In [49]:
# Without Categorical Features
model1_lgmb = lgb.train(params, d_train)

C:\Anaconda3\envs\classification2021\lib\site-packages\lightgbm\engine.py:148: UserWarning:

Found `n_estimators` in params. Will use it instead of argument



[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020216 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1524
[LightGBM] [Info] Number of data points in the train set: 428504, number of used features: 10
[LightGBM] [Info] Start training from score 0.219146


In [50]:
auc_lgbm1 = auc_from_pred(model1_lgmb, train, test)

In [51]:
# With Catgeorical Features
cate_features_name = [
    "MONTH",
    "DAY",
    "DAY_OF_WEEK",
    "AIRLINE",
    "DESTINATION_AIRPORT",
    "ORIGIN_AIRPORT",
]
model2_lgmb = lgb.train(params, d_train, categorical_feature=cate_features_name)

C:\Anaconda3\envs\classification2021\lib\site-packages\lightgbm\engine.py:148: UserWarning:

Found `n_estimators` in params. Will use it instead of argument

C:\Anaconda3\envs\classification2021\lib\site-packages\lightgbm\basic.py:1705: UserWarning:

categorical_feature in Dataset is overridden.
New categorical_feature is ['AIRLINE', 'DAY', 'DAY_OF_WEEK', 'DESTINATION_AIRPORT', 'MONTH', 'ORIGIN_AIRPORT']



[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005657 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1897
[LightGBM] [Info] Number of data points in the train set: 428504, number of used features: 10
[LightGBM] [Info] Start training from score 0.219146


In [52]:
auc_lgbm2 = auc_from_pred(model2_lgmb, train, test)

In [53]:
print(np.round(auc_lgbm1, 3))
print(np.round(auc_lgbm2, 3))

[0.981 0.785]
[0.998 0.775]


#### CatBoost

При настройке параметров для CatBoost трудно передать индексы для категориальных функций. 
Поэтому настроим параметры без передачи категориальных признаков и оценим две модели — одну с категориальными признаками, а другую без них. 


In [54]:
# Without Categorical features
clf1 = cb.CatBoostClassifier(
    eval_metric="AUC", depth=10, iterations=500, l2_leaf_reg=9, learning_rate=0.15
)
clf1.fit(train, y_train)

0:	total: 309ms	remaining: 2m 33s
1:	total: 486ms	remaining: 2m
2:	total: 664ms	remaining: 1m 50s
3:	total: 861ms	remaining: 1m 46s
4:	total: 1.05s	remaining: 1m 44s
5:	total: 1.26s	remaining: 1m 43s
6:	total: 1.46s	remaining: 1m 42s
7:	total: 1.67s	remaining: 1m 42s
8:	total: 1.9s	remaining: 1m 43s
9:	total: 2.11s	remaining: 1m 43s
10:	total: 2.28s	remaining: 1m 41s
11:	total: 2.47s	remaining: 1m 40s
12:	total: 2.64s	remaining: 1m 38s
13:	total: 2.81s	remaining: 1m 37s
14:	total: 2.97s	remaining: 1m 36s
15:	total: 3.12s	remaining: 1m 34s
16:	total: 3.28s	remaining: 1m 33s
17:	total: 3.43s	remaining: 1m 31s
18:	total: 3.59s	remaining: 1m 30s
19:	total: 3.76s	remaining: 1m 30s
20:	total: 3.94s	remaining: 1m 29s
21:	total: 4.1s	remaining: 1m 29s
22:	total: 4.25s	remaining: 1m 28s
23:	total: 4.4s	remaining: 1m 27s
24:	total: 4.56s	remaining: 1m 26s
25:	total: 4.72s	remaining: 1m 26s
26:	total: 4.89s	remaining: 1m 25s
27:	total: 5.07s	remaining: 1m 25s
28:	total: 5.23s	remaining: 1m 24s
29

In [55]:
auc_cbc1 = auc_from_proba(clf1, train, test)


In [56]:
# With Categorical features
cat_features_index = [0, 1, 2, 3, 4, 5, 6]

clf2 = cb.CatBoostClassifier(
    eval_metric="AUC",
    one_hot_max_size=31,
    depth=10,
    iterations=500,
    l2_leaf_reg=9,
    learning_rate=0.15,
)
clf2.fit(train, y_train, cat_features=cat_features_index)

0:	total: 911ms	remaining: 7m 34s
1:	total: 1.78s	remaining: 7m 24s
2:	total: 2.65s	remaining: 7m 18s
3:	total: 3.33s	remaining: 6m 52s
4:	total: 4s	remaining: 6m 36s
5:	total: 4.75s	remaining: 6m 31s
6:	total: 5.37s	remaining: 6m 18s
7:	total: 6.12s	remaining: 6m 16s
8:	total: 6.68s	remaining: 6m 4s
9:	total: 7.25s	remaining: 5m 55s
10:	total: 7.78s	remaining: 5m 46s
11:	total: 8.42s	remaining: 5m 42s
12:	total: 9.12s	remaining: 5m 41s
13:	total: 9.65s	remaining: 5m 34s
14:	total: 10.3s	remaining: 5m 32s
15:	total: 10.9s	remaining: 5m 30s
16:	total: 11.6s	remaining: 5m 28s
17:	total: 12.1s	remaining: 5m 24s
18:	total: 12.9s	remaining: 5m 25s
19:	total: 13.5s	remaining: 5m 24s
20:	total: 14.2s	remaining: 5m 23s
21:	total: 15s	remaining: 5m 26s
22:	total: 15.7s	remaining: 5m 25s
23:	total: 16.4s	remaining: 5m 25s
24:	total: 17s	remaining: 5m 23s
25:	total: 17.7s	remaining: 5m 23s
26:	total: 18.4s	remaining: 5m 22s
27:	total: 19.3s	remaining: 5m 25s
28:	total: 20.2s	remaining: 5m 28s
29:

In [57]:
auc_cbc2 = auc_from_proba(clf2, train, test)

In [63]:
print(np.round(auc_cbc1, 3))
print(np.round(auc_cbc2, 3))


[0.834 0.76 ]
[0.903 0.832]


#### Результаты:

<img src='https://miro.medium.com/max/3384/1*w05Hg2QZ5ioDi2OXdCCMiw.png' alignment='center'/>

### Интерпретация результатов из оригинальной статьи

Для оценки модели мы должны изучить производительность модели с точки зрения как скорости, так и точности.

Имея это в виду, CatBoost выходит победителем с максимальной точностью на тестовом наборе (0,816), минимальным переоснащением (точность как train, так и test близка) и минимальным временем прогнозирования и настройки. Но это произошло только потому, что мы рассмотрели категориальные переменные и настроили one_hot_max_size. Если мы не воспользуемся этими особенностями CatBoost, он окажется худшим исполнителем с точностью всего 0,752. Следовательно, мы узнали, что CatBoost работает хорошо только тогда, когда у нас есть категориальные переменные в данных, и мы правильно их настраиваем.

Нашим следующим исполнителем был XGBoost, который, как правило, работает хорошо. Его точность была довольно близка к CatBoost даже после игнорирования того факта, что у нас есть категориальные переменные в данных, которые мы преобразовали в числовые значения для его потребления. Однако единственная проблема с XGBoost заключается в том, что он слишком медленный.

Последнее место занимает Light GBM. Здесь важно отметить, что он плохо работал как с точки зрения скорости, так и точности при использовании cat_features. Причина, по которой он плохо работал, заключалась скорее всего в том, что он использует какое—то модифицированное среднее кодирование для категориальных данных, которое вызвало переобучение (точность train довольно высока-0,999 по сравнению с точностью теста). 
Однако, если мы используем его обычно, как XGBoost, он может достичь аналогичной (если не более высокой) точности с гораздо более высокой скоростью по сравнению с XGBoost

Наконец, важно заметить, что эти наблюдения верны для данного конкретного набора данных и могут оставаться или не оставаться верными для других наборов данных. Однако в целом верно то, что XGBoost работает медленнее, чем два других алгоритма.

## GradientBoostingClassifier

In [69]:
from sklearn.ensemble import GradientBoostingClassifier

model_sk = GradientBoostingClassifier()
model_sk.fit(train, y_train)

GradientBoostingClassifier()

In [70]:
auc_sk = auc_from_proba(model_sk, train, test)
print(np.round(auc_sk, 3))

[0.7   0.696]


# Воспроизведенные результаты

|                      | XGBoost                                                                     | LightGBM without categorial features                                  | LightGBM with categorial features | CatBoost without categorial features                                                             | CatBoost with categorial features | SKLearn                                            |
| -------------------- | --------------------------------------------------------------------------- | --------------------------------------------------------------------- | --------------------------------- | ------------------------------------------------------------------------------------------------ | --------------------------------- | -------------------------------------------------- |
| Params               | max_depth=50,min_child_weight=1,    n_estimators=200,    learning_rate=0.16 |  max_depth=50, learning_rate=0.1, num_leaves=900, n_estimators=300  |                                   | one_hot_max_size=31,    depth=10,    iterations=500,    l2_leaf_reg=9,    learning_rate=0.15                                     || max_depth=10, n_estimators=200, learning_rate=0.16 |
| Train AUC Score      | 1                                                                           | 0.981                                                                 | 0.998                             | 0.834                                                                                            | 0.903                             |                      0.7                              |
| Test AUC Score       | 0.789                                                                       | 0.785                                                                 | 0.775                             | 0.76                                                                                             | 0.832                             |                    0.696                                |
| Training Time, sec   | 505                                                                         | 22                                                                    | 33                                | 89                                                                                               | 332                               | 82                                                 |
| Prediction Time, sec | 16                                                                          | 10                                                                    | 20                                | 2                                                                                                | 13                                | 2                                                  |

### Ссылки:

- [Документация sklearn](https://scikit-learn.org/stable/modules/ensemble.html) для реализации ансамблей
- [AdaBoost](https://www.cs.princeton.edu/courses/archive/spr07/cos424/papers/boosting-survey.pdf)
- [Модификации AdaBoost](https://cseweb.ucsd.edu/~yfreund/papers/boostingexperiments.pdf)
- [Статья про введение в градиентные бустинги](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/)
- [XGBoost, LightGBM or CatBoost — which boosting algorithm should I use?](https://medium.com/riskified-technology/xgboost-lightgbm-or-catboost-which-boosting-algorithm-should-i-use-e7fda7bb36bc)
- [Оригинальная статья про сравнение бустингов](https://towardsdatascience.com/catboost-vs-light-gbm-vs-xgboost-5f93620723db)
- [ODS Тема 10. Градиентный бустинг](https://habr.com/ru/company/ods/blog/327250/#2-gbm-algoritm)
- [Лукция. Машинное обучение. Линейные композиции, бустинг. К.В. Воронцов, ШАД, Яндекс.](https://www.youtube.com/watch?v=5QrWW_ZlP9w&list=PLJOzdkh8T5krxc4HsHbB8g8f0hu7973fK&index=15)
